In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import json
import os
from PIL import ImageDraw
from datetime import datetime as dt
import gc
import collections

from IPython.display import FileLink

In [ ]:
with open('/home/ubuntu/iwildcam2020_megadetector_results.json', encoding='utf-8') as json_file:
    megadetector_results =json.load(json_file)
    
megadetector_results.keys()

In [ ]:
megadetector_results_df = pd.DataFrame(megadetector_results["images"])
megadetector_results_df.head()

In [ ]:
def draw_bboxs(detections_list, im):
    """
    detections_list: list of set includes bbox.
    im: image read by Pillow.
    """
    
    for detection in detections_list:
        x1, y1,w_box, h_box = detection["bbox"]
        ymin,xmin,ymax, xmax=y1, x1, y1 + h_box, x1 + w_box
        draw = ImageDraw.Draw(im)
        
        imageWidth=im.size[0]
        imageHeight= im.size[1]
        (left, right, top, bottom) = (xmin * imageWidth, xmax * imageWidth,
                                      ymin * imageHeight, ymax * imageHeight)
        
        draw.line([(left, top), (left, bottom), (right, bottom),
               (right, top), (left, top)], width=4, fill='Red')

In [ ]:
data_num = 8857
im = Image.open("/home/ubuntu/train/" + megadetector_results_df.loc[data_num]['id'] + ".jpg")
im = im.resize((480,270))
draw_bboxs(megadetector_results_df.loc[data_num]['detections'], im)

In [ ]:
plt.imshow(im)

In [ ]:
def get_data(x):
    if x == []:
        return 0
    return len(x)

megadetector_results_df["detected_num"] = megadetector_results_df.loc[:, "detections"].map(get_data)

In [ ]:
ax = sns.countplot(x="detected_num", data=megadetector_results_df)
ax.set(ylabel='count')
#ax.set(ylim=(0,80000))
plt.title('distribution of count per animals each data of train')

In [ ]:
megadetector_results_df = megadetector_results_df[megadetector_results_df['detected_num'] < 2]
megadetector_results_df = megadetector_results_df.rename(columns={'id': 'image_id'})
megadetector_results_df.head()

In [ ]:
with open('/home/ubuntu/iwildcam2020_train_annotations.json') as json_file:
    train_annotations_json = json.load(json_file)

In [ ]:
train_annotations_json.keys()

In [ ]:
df_annotations = pd.DataFrame(train_annotations_json["annotations"])

In [ ]:
df_annotations.head()

In [ ]:
df_images = pd.DataFrame(train_annotations_json["images"])

In [ ]:
df_images.head()

In [ ]:
df_categories = pd.DataFrame(train_annotations_json["categories"])

In [ ]:
df_categories.head()

In [ ]:
print(f"There are {len(df_categories) - 1} species")

In [ ]:
df_annotations.to_csv("iwildcam2020_train_annotations_annotations.csv", index=False)
df_images.to_csv("iwildcam2020_train_annotations_images.csv", index=False)
df_categories.to_csv("iwildcam2020_train_annotations_categories.csv", index=False)

In [ ]:
with open('/home/ubuntu/iwildcam2020_test_information.json') as json_file:
    test_information_json = json.load(json_file)

In [ ]:
test_information_json.keys()

In [ ]:
df_images_test = pd.DataFrame(test_information_json["images"])
df_images_test.head()

In [ ]:
df_categories_test = pd.DataFrame(test_information_json["categories"])
df_categories_test.head()

In [ ]:
df_images_test.to_csv("iwildcam2020_train_annotations_images_test.csv", index=False)
df_categories_test.to_csv("iwildcam2020_train_annotations_categories_test.csv", index=False)

In [ ]:
month_year = df_images['datetime'].map(lambda str: str[2:7])
labels_month_year = sorted(list(set(month_year)))

month_year_test = df_images_test['datetime'].map(lambda str: str[2:7])

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(30,7))
ax = plt.subplot(1,2,1)
ax = plt.title('Count of train data per month & year')
ax = sns.countplot(month_year, order=labels_month_year)
ax.set(xlabel='YY-mm', ylabel='count')
ax.set(ylim=(0,50000))

ax = plt.subplot(1,2,2)
ax = plt.title('Count of test data per month & year')

ax = sns.countplot(month_year_test, order=labels_month_year)
ax.set(xlabel='YY-mm', ylabel='count')
ax.set(ylim=(0,50000))

In [ ]:
labels_month = sorted(list(set(df_images['datetime'].map(lambda str: str[5:7]))))

In [ ]:
ax = plt.subplot(1,2,1)
plt.title('Count of train data per month')
ax = sns.countplot(df_images['datetime'].map(lambda str: str[5:7] ), order=labels_month)
ax.set(xlabel='mm', ylabel='count')
ax.set(ylim=(0,55000))

ax = plt.subplot(1,2,2)
plt.title('Count of test data per month')
ax = sns.countplot(df_images_test['datetime'].map(lambda str: str[5:7] ), order=labels_month)
ax.set(xlabel='mm', ylabel='count')
ax.set(ylim=(0,55000))

In [ ]:
train_taken_hour = df_images['datetime'].map(lambda x: dt.strptime(x, '%Y-%m-%d %H:%M:%S.%f').hour)
test_taken_hour = df_images_test['datetime'].map(lambda x: dt.strptime(x, '%Y-%m-%d %H:%M:%S.%f').hour)

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(20,7))
ax = plt.subplot(1,2,1)
plt.title('Count of train data per hour')
ax = sns.countplot(train_taken_hour)
ax.set(xlabel='hour', ylabel='count')
ax.set(ylim=(0,20000))

ax = plt.subplot(1,2,2)
plt.title('Count of test data per hour')
ax = sns.countplot(test_taken_hour)
ax.set(xlabel='hour', ylabel='count')
ax.set(ylim=(0,20000))

In [ ]:
train_taken_phase = train_taken_hour.map(lambda x: "daytime" if x >= 6 and x < 18 else "night")
test_taken_phase = test_taken_hour.map(lambda x: "daytime" if x >= 6 and x < 18 else "night")

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(20,7))
ax = plt.subplot(1,2,1)
plt.title('Count of train data per phase')
ax = sns.countplot(train_taken_phase, order=["daytime", "night"])
ax.set(xlabel='phase', ylabel='count')
ax.set(ylim=(0,200000))

ax = plt.subplot(1,2,2)
plt.title('Count of test data per phase')
ax = sns.countplot(test_taken_phase, order=["daytime", "night"])
ax.set(xlabel='phase', ylabel='count')
ax.set(ylim=(0,200000))

In [ ]:
del train_taken_phase
del test_taken_phase
del train_taken_hour
del test_taken_hour
gc.collect()

In [ ]:
fig = plt.figure(figsize=(15, 9))
ax = sns.distplot(df_categories['count'][1:])
ax.set(ylim=(0,0.00005))
ax.set(xlabel='count')
plt.title('distribution of number of data per id zoomed')

In [ ]:
labels_location_train = sorted(list(set(df_images['location'])))
labels_location_test = sorted(list(set(df_images_test['location'])))
labels_location = labels_location_train + labels_location_test

In [ ]:
loc_cat_df_test = pd.merge(df_images, df_annotations, left_on='id', right_on='image_id', how = "inner").loc[:,["location", "category_id", "image_id"]]
loc_cat_df_test.head()

In [ ]:
loc_cat_dict = {}
for loc in set(loc_cat_df_test["location"]):
    loc_cat_dict[loc] = list(set(loc_cat_df_test[loc_cat_df_test["location"] == loc]["category_id"]))   
loc_cat_matrix = np.zeros([loc_cat_df_test["location"].max()+1, loc_cat_df_test["category_id"].max()+1])
loc_cat_matrix.shape


In [ ]:
for loc in loc_cat_dict.keys():
    for cat in loc_cat_dict[loc]:
        loc_cat_matrix[loc, cat] = 1

In [ ]:
fig = plt.figure(figsize=(15, 15))
ax = sns.heatmap(loc_cat_matrix)
ax.set(xlabel='category', ylabel='location')
plt.title('Relation between animal categories and locations')

In [ ]:
df_annotations[2429:2440]

In [ ]:
fig = plt.figure(figsize=(25, 16))
#for i,im_path in enumerate(df_annotations[df_annotations.loc[:,"count"] == 3].loc[:,"image_id"][2429:2440]):
for i,im_path in enumerate(df_annotations.loc[:,"image_id"][2429:2440]):
    ax = fig.add_subplot(4, 4, i+1, xticks=[], yticks=[])
    im = Image.open("/home/ubuntu/train/" + im_path + ".jpg")
    im = im.resize((480,270))
    plt.imshow(im)

In [ ]:
c = collections.Counter(df_annotations.loc[:,"count"])
animals_in_pict = list(c.keys())
freq = list(c.values())
k = zip(animals_in_pict,freq)

In [ ]:
animals_in_pict_df = pd.DataFrame(sorted(k),columns=['num_of_animals','freq'])

In [ ]:
animals_in_pict_df

In [ ]:
species_morethan_10 = list(set(df_annotations[df_annotations.loc[:,"count"] >= 10].loc[:,"category_id"]))
df_categories[df_categories.loc[:,"id"].isin(species_morethan_10)].loc[:,["id", "name"]]

In [ ]:
species_morethan_30 = list(set(df_annotations[df_annotations.loc[:,"count"] >= 30].loc[:,"category_id"]))
df_categories[df_categories.loc[:,"id"].isin(species_morethan_30)].loc[:,["id", "name"]]

In [ ]:
df_annotations[(df_annotations.loc[:,"category_id"] == 2) & (df_annotations.loc[:,"count"] >= 30)].head()

In [ ]:
fig = plt.figure(figsize=(25, 16))
#for i,im_path in enumerate(df_annotations[df_annotations.loc[:,"count"] == 3].loc[:,"image_id"][2429:2440]):
for i,im_path in enumerate(df_annotations.loc[:,"image_id"][149674:149678]):
    ax = fig.add_subplot(4, 4, i+1, xticks=[], yticks=[])
    im = Image.open("/home/ubuntu/train/" + im_path + ".jpg")
    im = im.resize((480,270))
    plt.imshow(im)

In [ ]:
df_annotations[(df_annotations.loc[:,"category_id"] == 71) & (df_annotations.loc[:,"count"] >= 30)].head()

In [ ]:
jpeg ='/home/ubuntu/train/8897a9f8-21bc-11ea-a13a-137349068a90.jpg'
im = Image.open(jpeg)
im = im.resize((480,270))
plt.imshow(im)

In [ ]:
!pip install tensorflow-gpu==1.14.0
!pip install keras==2.2.4
!pip install git+https://github.com/qubvel/efficientnet
!ip install -U segmentation-models

import cv2
from copy import deepcopy
import efficientnet.keras as efn 
import glob
from IPython.display import Image
import json
import keras
from keras import backend as K
import keras
#import keras.backend as K
from keras.optimizers import Adam
from keras.applications.densenet import DenseNet201
from keras.layers import Dense, Flatten, Activation, Dropout, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers, applications
from keras.models import Model, load_model
from keras.callbacks import Callback, ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.utils import Sequence
import matplotlib.pyplot as plt
import multiprocessing
import numpy as np
from numpy.random import seed
import os
import random
from sklearn.metrics import precision_recall_curve, auc
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook as tqdm

seed(10)

%matplotlib inline

In [ ]:
train_anns_df = df_annotations[['image_id','category_id']]
train_img_df = df_images[['id', 'file_name']].rename(columns={'id':'image_id'})
df_train_file_cat = pd.merge(train_img_df, train_anns_df, on='image_id')
df_train_file_cat['category_id']=df_train_file_cat['category_id'].astype(str)
df_train_file_cat.head()


In [ ]:
df_train_file_cat = pd.merge(megadetector_results_df, df_train_file_cat, on='image_id', how='inner')

fig = plt.figure(figsize=(15, 4))
ax = sns.countplot(x="detected_num", data=megadetector_results_df)
ax.set(ylabel='count')
#ax.set(ylim=(0,80000))
plt.title('distribution of count per animals each data of train')

In [ ]:
batch_size = 128 #changed from 256
img_size = 96
lr = 0.0005 #changed from 0.0008
nb_classes = 267
nb_epochs = 25 #changed from 6, then 15

In [ ]:
checkpoint_filepath = "/home/ubuntu/epoch_checkpoint"
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_acc',
    mode='max',
    save_best_only=False)

In [ ]:
%%time

train_datagen=ImageDataGenerator(rescale=1./255, 
    validation_split=0.25,
    horizontal_flip = True,    
    zoom_range = 0.3,
    width_shift_range = 0.3,
    height_shift_range=0.3
    )

train_generator=train_datagen.flow_from_dataframe(    
    dataframe=df_train_file_cat[:50000],    
    directory="/home/ubuntu/train",
    x_col="file_name",
    y_col="category_id",
    batch_size=batch_size,
    shuffle=True,
    classes = [ str(i) for i in range(nb_classes)],
    class_mode="categorical",    
    target_size=(img_size,img_size))

test_datagen = ImageDataGenerator(rescale=1./255)

valid_generator=test_datagen.flow_from_dataframe(    
    dataframe=df_train_file_cat[50000:],    
    directory="/home/ubuntu/train",
    x_col="file_name",
    y_col="category_id",
    batch_size=batch_size,
    shuffle=True,
    classes = [ str(i) for i in range(nb_classes)],
    class_mode="categorical",  
    target_size=(img_size,img_size))

In [ ]:
from keras import backend as K
import efficientnet.keras as efn 

def get_model():
    K.clear_session()
    base_model =  efn.EfficientNetB3(weights='imagenet', include_top=False, pooling='avg', input_shape=(img_size, img_size, 3))
    x = base_model.output
    predictions = Dense(nb_classes, activation="softmax")(x)
    return Model(inputs=base_model.input, outputs=predictions)

model = get_model()
model.compile(optimizers.Adam(lr=lr, decay=1e-6),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
early = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=1, mode='auto')

In [ ]:
%%time
history = model.fit_generator(generator=train_generator,  
                                    steps_per_epoch=5,
                                    validation_data=valid_generator, 
                                    validation_steps=2,
                                    epochs=nb_epochs,
                                    callbacks = [early], #model_checkpoint_callback
                                    verbose=2)

In [ ]:
history_df = pd.DataFrame(history.history)
history_df[['loss', 'val_loss']].plot()
history_df[['acc', 'val_acc']].plot()

In [ ]:
import gc
del train_datagen, train_generator
gc.collect()

In [ ]:
sam_sub_df = pd.read_csv('/home/ubuntu/sample_submission.csv')
sam_sub_df["file_name"] = sam_sub_df["Id"].map(lambda str : str + ".jpg")
sam_sub_df.head()


In [ ]:
%%time

test_generator = test_datagen.flow_from_dataframe(      
    
        dataframe=sam_sub_df,    
    
        directory = "/home/ubuntu/test",    
        x_col="file_name",
        target_size = (img_size,img_size),
        batch_size = 1,
        classes = [ str(i) for i in range(nb_classes)],
        shuffle = False,
        class_mode = None
        )

In [ ]:
%%time
test_generator.reset()
predict=model.predict_generator(test_generator, steps = len(test_generator.filenames))
predicted_class_indices=np.argmax(predict,axis=1)
sam_sub_df["Category"] = predicted_class_indices
sam_sub_df = sam_sub_df.loc[:,["Id", "Category"]]
sam_sub_df.to_csv("submission_2.csv",index=False)